In [11]:
import numpy as np
import pandas as pd
import yfinance as yf
import pandas_ta as ta
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Flatten, Permute, Reshape
import matplotlib.pyplot as plt

def import_data(stock, startDate):
    data = yf.download(stock, start=startDate, interval='1d')
    data.columns = data.columns.get_level_values(0)
    data['EMA_50'] = ta.ema(data['Close'], length=50)
    data['EMA_200'] = ta.ema(data['Close'], length=200)
    data['SMA_50'] = ta.sma(data['Close'], length=50)
    data['SMA_200'] = ta.sma(data['Close'], length=200)
    data['RSI'] = ta.rsi(data['Close'], length=14)
    data['Pct_Change_5D'] = data['Close'].pct_change(periods=5).shift(-5)  # Percent change over 5 days
    data.dropna(inplace=True)  # Remove any rows with NaN values
    return data

In [12]:
# import numpy as np
# import pandas as pd
# import matplotlib.pyplot as plt
# from sklearn.metrics import mean_absolute_error, mean_squared_error
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import MinMaxScaler
# from tensorflow.keras.models import Sequential
# from keras.layers import LSTM, Dense, Dropout, AdditiveAttention, Permute, Reshape, Multiply, Flatten

# data = import_data('SPY', '2016-01-01')
# data.columns = data.columns.get_level_values(0)

# # Rename the columns to their appropriate names
# data.rename(columns={'Adj Close': 'Adj_Close', 'Close': 'Close', 'High': 'High', 'Low': 'Low',
#                    'Open': 'Open', 'Volume': 'Volume'}, inplace=True)
# data = data.dropna()
# print(data)

# # scalers = {}
# # for column in ['Close', 'RSI', 'SMA_50', 'SMA_200', 'EMA_50', 'EMA_200', 'Volume']:
# #     scaler = MinMaxScaler(feature_range=(0, 1))
# #     data[column] = scaler.fit_transform(data[column].values.reshape(-1, 1))
# #     scalers[column] = scaler
# step = 60
# X, Y = [], []
# # import all SPY tickers
# sp500_url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
# sp500_table = pd.read_html(sp500_url)
# TICKERS = sp500_table[0]['Symbol'].tolist()
# # import each ticker's data
# for ticker in TICKERS:
#     dataset = import_data(ticker, '2016-01-01')
#     # scale all columns in data
#     for column in ['RSI', 'SMA_50', 'SMA_200', 'EMA_50', 'EMA_200', 'Volume']:
#         scaler = MinMaxScaler(feature_range=(0, 1))
#         dataset[column] = scaler.fit_transform(dataset[column].values.reshape(-1, 1))
#     for i in range(len(dataset) - step - 6):
#         features = dataset[['Close', 'RSI', 'SMA_50', 'SMA_200', 'EMA_50', 'EMA_200', 'Volume']].iloc[i:(i + step)].values
        
#         current_price = dataset['Close'].iloc[i + step]
#         future_price = dataset['Close'].iloc[i + step + 5]
#         percent_change = ((future_price - current_price) / current_price) * 100
        
#         X.append(features)
#         Y.append(percent_change)
# X, Y = np.array(X), np.array(Y)

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# model = Sequential()
# model.add(LSTM(50, return_sequences=True, input_shape=(X.shape[1], X.shape[2])))
# model.add(Dropout(0.2))
# model.add(LSTM(50, return_sequences=True))
# # attention = AdditiveAttention(name='attention_weight')

# model.add(Permute((2, 1))) 
# model.add(Reshape((-1, X_train.shape[1])))
# # attention_result = attention([model.output, model.output])
# # multiply_layer = Multiply()([model.output, attention_result])

# model.add(Permute((2, 1))) 
# model.add(Reshape((-1, 50)))
# model.add(Flatten())
# model.add(Dense(1))

# model.compile(optimizer='adam', loss='mean_squared_error')

# # model.fit(X_train, y_train, epochs=100, batch_size=25, validation_split=0.2)
# model.fit(X_train, y_train, epochs=10, batch_size=25, validation_split=0.2)

# predictions = model.predict(X_test)
# predictions = predictions.reshape(-1, 1)

# print(predictions)

# # Evaluate the model
# mae = mean_absolute_error(y_test, predictions)
# rmse = mean_squared_error(y_test, predictions, squared=False)

# print("Mean Absolute Error: ", mae)
# print("Root Mean Square Error: ", rmse)  

# plt.plot(y_test, label='Original Data')
# plt.plot(np.arange(len(y_train) + step, len(y_train) + len(predictions) + step), predictions, label='Predicted Data', color='red')
# plt.xlabel('Time')
# plt.ylabel('Stock Price')
# plt.legend()
# plt.show()

In [13]:
# from backtesting import Strategy, Backtest

# def get_parameters(df):
#     # Define the signal function that will be used in the strategy
#     def SIGNAL():
#         return df['pc_change']
    
#     class MA_Crossover_RSI_Strategy(Strategy):
#         mysize = 1  # Trade size (100% of available cash)
#         trade_age = 0
#         threshold = 1

#         def init(self):
#             # Initialize the signal
#             self.signal1 = self.I(SIGNAL)

#         def next(self):
#             # If no open position and a signal occurs, enter a buy order
#             if abs(self.signal1) > self.threshold and not self.position:
#                 if self.signal1 > 0:
#                     self.buy(size=self.mysize)
#                 else:
#                     self.sell(size=self.mysize)
#                 # print(f"BUY at {entry_price}, Stop Loss at {stop_loss}")

#             # If signal is 5 days old, close position
#             if self.position:
#                 if self.trade_age >= 5:
#                     self.position.close()
#                     self.trade_age = 0
#                 else:
#                     self.trade_age += 1

#     # Run the backtest
#     bt = Backtest(df, MA_Crossover_RSI_Strategy, cash=10000, margin=1/30, commission=0.01)
#     results = bt.run()

#     return results

In [14]:
def prepare_lstm_data(data, feature_columns, target_column, time_steps=60):
    X = []
    Y = []
    for i in range(time_steps, len(data) - 5):  # Ensure we have future data for Y
        X.append(data[feature_columns].iloc[i - time_steps:i].values)
        Y.append(data[target_column].iloc[i])
    return np.array(X), np.array(Y)

In [ ]:
sp500_url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
sp500_table = pd.read_html(sp500_url)
TICKERS = sp500_table[0]['Symbol'].tolist()

# Select a subset for faster testing
TICKERS = TICKERS[:10]  # Adjust as needed

# Prepare data for training
feature_columns = ['Close', 'EMA_50', 'EMA_200', 'SMA_50', 'SMA_200', 'RSI']
X_train = []
Y_train = []
X_test = []
Y_test = []
time_steps = 60  # Number of time steps for LSTM
scaler = MinMaxScaler()

for ticker in TICKERS:
    # print(f"Processing {ticker}...")
    try:
        data = import_data(ticker, startDate='2023-01-01')
        data[feature_columns] = scaler.fit_transform(data[feature_columns])
        X, Y = prepare_lstm_data(data, feature_columns, 'Pct_Change_5D', time_steps)
        if len(X) > 0 and len(Y) > 0:
            length = len(X)
            index = 0.8*length
            X_train.extend(X[:index])
            Y_train.extend(Y[:index])
            X_test.extend(X[index:])
            Y_test.extend(Y[index:])

    except Exception as e:
        print(f"Skipping {ticker} due to an error: {e}")

X_train = np.array(X_train)
Y_train = np.array(Y_train)

Processing MMM...


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

Skipping MMM due to an error: slice indices must be integers or None or have an __index__ method
Processing AOS...



[*********************100%%**********************]  1 of 1 completed

Skipping AOS due to an error: slice indices must be integers or None or have an __index__ method
Processing ABT...


...

Skipping AMD due to an error: slice indices must be integers or None or have an __index__ method
Processing AES...



[*********************100%%**********************]  1 of 1 completed

Skipping AES due to an error: slice indices must be integers or None or have an __index__ method
Processing AFL...



[*********************100%%**********************]  1 of 1 completed

Skipping AFL due to an error: slice indices must be integers or None or have an __index__ method
Processing A...
Skipping A due to an error: slice indices must be integers or None or have an __index__ method


In [16]:
model = Sequential()
model.add(LSTM(50, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dropout(0.2))
model.add(LSTM(50))
model.add(Dense(1))

model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(X_train, Y_train, epochs=50, batch_size=32)

IndexError: tuple index out of range

In [ ]:
def backtest(model, tickers, start_date, feature_columns, time_steps=60):
    capital = 10000  # Starting capital
    equity_curve = [capital]
    current_capital = capital

    # Create a combined dataframe for all tickers with predicted values
    combined_data = []

    for ticker in tickers:
        print(f"Backtesting {ticker}...")
        try:
            data = import_data(ticker, startDate=start_date)
            data[feature_columns] = scaler.transform(data[feature_columns])
            X_test, _ = prepare_lstm_data(data, feature_columns, 'Pct_Change_5D', time_steps)

            # Predict and add to combined data
            data['Predicted'] = np.nan
            for i in range(time_steps, len(data) - 5):
                pred = model.predict(X_test[i - time_steps].reshape(1, time_steps, -1), verbose=0)
                data.loc[data.index[i], 'Predicted'] = pred
            combined_data.append(data)
        except Exception as e:
            print(f"Skipping {ticker} due to an error: {e}")

    print(data.index[-1])
    for i in range(len(combined_data)):
        combined_data[i].index = combined_data[i].index.tz_localize(None)
    # Simulate backtesting over the given period
    for date in pd.date_range(start=start_date, end=data.index[-1]):
        best_stock = None
        max_pred = -np.inf

        # Find the stock with the highest predicted percent change for the given day
        for data in combined_data:
            if date in data.index and not pd.isna(data.loc[date, 'Predicted']):
                if data.loc[date, 'Predicted'] > max_pred:
                    max_pred = data.loc[date, 'Predicted']
                    best_stock = data

        # "Buy" the stock with the highest predicted change
        if best_stock is not None:
            buy_price = best_stock.loc[date, 'Close']
            future_date = date + pd.Timedelta(days=5)
            if future_date in best_stock.index:
                sell_price = best_stock.loc[future_date, 'Close']
                profit = (sell_price - buy_price) / buy_price * current_capital
                current_capital += profit
                equity_curve.append(current_capital)

    return equity_curve

In [ ]:
from sklearn.metrics import roc_auc_score
# start_date = '2022-06-01'
# equity_curve = backtest(model, TICKERS, start_date, feature_columns, time_steps)

# # Plot the equity curve
# plt.plot(equity_curve)
# plt.title('Equity Curve')
# plt.xlabel('Time')
# plt.ylabel('Equity ($)')
# plt.show()

y_pred = model.predict(X_test)
roc_auc = roc_auc_score(Y_test, y_pred)